In [ ]:
import opendatasets as od
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import ipywidgets as widgets
from io import BytesIO
import pickle

In [ ]:
TRAIN_DIR = './new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
VALIDATION_DIR = './new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

In [ ]:
Diseases_classes = os.listdir(TRAIN_DIR)
print("\nTotal number of classes are: ", len(Diseases_classes))

In [ ]:
def check_for_invalid_files(dataset_dir):
    # List to store paths of missing files
    missing_files = []
    
    # Iterate through each plant class in the dataset
    for plant_class in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, plant_class)
        
        if os.path.isdir(class_dir):  # Check if it's a directory
            for filename in os.listdir(class_dir):
                file_path = os.path.join(class_dir, filename)
                
                # Check if the file exists
                if not os.path.exists(file_path):
                    print(f"File not found: {file_path}")
                    missing_files.append(file_path)  # Add missing file to the list
    
    return missing_files

# Check the training dataset
print("Checking training dataset...")
missing_train_files = check_for_invalid_files(TRAIN_DIR)

# Check the validation dataset
print("\nChecking validation dataset...")
missing_valid_files = check_for_invalid_files(VALIDATION_DIR)

# Report results
if missing_train_files:
    print(f"\nTotal missing files in training dataset: {len(missing_train_files)}")
else:
    print("\nNo missing files in training dataset.")

if missing_valid_files:
    print(f"\nTotal missing files in validation dataset: {len(missing_valid_files)}")
else:
    print("\nNo missing files in validation dataset.")

print("\nFinished checking both datasets.")

In [ ]:
fig, axes = plt.subplots(1, 6, figsize=(14, 3))
fig.suptitle('Plant Disease Images', fontsize=16)

# Loop through the first 6 classes in Diseases_classes and plot an image from each
for ii, disease in enumerate(Diseases_classes[:6]):
    # Set the directory for the current disease class
    dir = f'{TRAIN_DIR}/{disease}'
    # Load the first image in the directory
    img = tf.keras.preprocessing.image.load_img(dir + '/' + os.listdir(dir)[0])
    # Display the image in the subplot
    axes[ii].imshow(img)
    axes[ii].set_title(f'{disease}')

plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt

plt.figure(figsize=(60, 60), dpi=200)
cnt = 0
plant_names = []
tot_images = 0

for i in Diseases_classes:
    cnt += 1
    plant_names.append(i)
    plt.subplot(7, 7, cnt)
    
    # Get only valid image files by filtering out hidden folders like .ipynb_checkpoints
    image_path = [f for f in os.listdir(os.path.join(TRAIN_DIR, i)) if not f.startswith('.')]
    
    print("The Number of Images in " + i + ":", len(image_path))
    tot_images += len(image_path)
    
    # Load and display the first image if there are any images in the folder
    if image_path:
        img_show = plt.imread(os.path.join(TRAIN_DIR, i, image_path[0]))
        plt.imshow(img_show)
        plt.xlabel(i, fontsize=30)
        plt.xticks([])
        plt.yticks([])
    
print("\nTotal Number of Images in Directory:", tot_images)


In [ ]:
def train_val_datasets():
    train_dataset = tf.keras.utils.image_dataset_from_directory( 
        directory=TRAIN_DIR,
        batch_size=32,
        image_size=(256, 256),
		label_mode= "int",
        color_mode= "rgb",
    ) 
    
    validation_dataset = tf.keras.utils.image_dataset_from_directory( 
        directory=VALIDATION_DIR,
        batch_size=32,
        image_size=(256, 256),
		label_mode="int",
        color_mode= "rgb",
    ) 
    
    return train_dataset, validation_dataset

In [ ]:
train_dataset, validation_dataset = train_val_datasets()
print(f"Images of train dataset have shape: {train_dataset.element_spec[0].shape}")
print(f"Labels of train dataset have shape: {train_dataset.element_spec[1].shape}")
print(f"Images of validation dataset have shape: {validation_dataset.element_spec[0].shape}")
print(f"Labels of validation dataset have shape: {validation_dataset.element_spec[1].shape}")   

In [ ]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Rescaling(1./255, input_shape=(256, 256, 3)),
        
        # Conv Block 1
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.2),
        
        # Conv Block 2
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.2),
        
        # Conv Block 3
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.2),
        
        # Conv Block 4 (Tambahan)
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.2),
        
        # Fully Connected Layer
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(38, activation='softmax')  # 38 class output
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


## Jalankan Salah Satu Aja

### Untuk Training Dari Awal:

In [ ]:
model = create_model()

### Untuk Training Import Model yang Udah Ada:

In [ ]:
model = tf.keras.models.load_model('plant_disease_model5.h5')
model.load_weights('plant_disease_weights5.h5')

In [ ]:
print(f'Input shape: {model.input_shape}')
print(f'Output shape: {model.output_shape}')

In [ ]:
model.summary()

In [ ]:
for images, labels in train_dataset.take(1):
	example_batch_images = images
	example_batch_labels = labels
	
try:
	model.evaluate(example_batch_images, example_batch_labels, verbose=False)
except:
	print("Your model is not compatible with the dataset you defined earlier. Check that the loss function, last layer and label_mode are compatible with one another.")
else:
	predictions = model.predict(example_batch_images, verbose=False)
	print(f"predictions have shape: {predictions.shape}")

In [ ]:
# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=3, restore_best_weights=True, monitor='val_loss'
)

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=4,
    callbacks=[early_stopping]
)

In [ ]:
uploader = widgets.FileUpload(accept="image/*", multiple=True, description='Upload Gambar', button_style='primary')
out = widgets.Output()
display(uploader)
display(out)

def file_predict(filename, file, out):
    image = tf.keras.utils.load_img(file, target_size=(256, 256))
    image = tf.keras.utils.img_to_array(image)
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image, verbose=0)[0]
    confidence_score = np.max(prediction) * 100
    
    with out:

        print(f'\nmodel output: {prediction}')
        
        prediction_index = np.argmax(prediction)
        
        dataset = tf.keras.utils.image_dataset_from_directory(TRAIN_DIR)

        # Ambil nama kelas
        classes = dataset.class_names
        
        predicted_class = classes[prediction_index]
        
        print(f'{filename} is predicted as {predicted_class} with a confidence score of {confidence_score:.2f}%')


def on_upload_change(change):
    items = change.new
    for item in items: # Loop if there is more than one file uploaded  
        file_jpgdata = BytesIO(item.content)
        file_predict(item.name, file_jpgdata, out)
        
uploader.observe(on_upload_change, names='value')

In [ ]:
# Simpan model
model.save('plant_disease_model6.h5')

# Jika ada weights/bobot tertentu yang ingin disimpan
model.save_weights('plant_disease_weights6.h5')

import pickle
with open('training_history6.pkl', 'wb') as file:
    pickle.dump(history.history, file)

In [ ]:
def plot_loss_acc(history):
  '''Plots the training and validation loss and accuracy from a history object'''
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  fig, ax = plt.subplots(1,2, figsize=(12, 6))
  ax[0].plot(epochs, acc, 'b-', label='Training accuracy')
  ax[0].plot(epochs, val_acc, 'r-', label='Validation accuracy')
  ax[0].set_title('Training and validation accuracy')
  ax[0].set_xlabel('epochs')
  ax[0].set_ylabel('accuracy')
  ax[0].legend()

  ax[1].plot(epochs, loss, 'b-', label='Training Loss')
  ax[1].plot(epochs, val_loss, 'r-', label='Validation Loss')
  ax[1].set_title('Training and validation loss')
  ax[1].set_xlabel('epochs')
  ax[1].set_ylabel('loss')
  ax[1].legend()

  plt.show()

plot_loss_acc(history)

In [ ]:
import json

# Simpan history ke file JSON
with open('training_history5.json', 'w') as file:
    json.dump(history.history, file)

In [ ]:
import json

# Untuk memuat kembali history dari file
with open('training_history5.json', 'r') as file:
    history = json.load(file)

In [ ]:
import pickle

with open('training_history5.pkl', 'rb') as file:
    history = pickle.load(file)

In [ ]:
loss, accuracy = model.evaluate(validation_dataset, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * accuracy))